In [1]:
cd /Users/jeremy_bon/Documents/Perso/Ploutos/ploutos


/Users/jeremy_bon/Documents/Perso/Ploutos/ploutos


/Users/jeremy_bon/Library/Caches/pypoetry/virtualenvs/ploutos-syHIIjPM-py3.12/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:

from pandas import Timestamp

from dotenv import load_dotenv
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from utils.sheet_migration import SheetMigration
load_dotenv(dotenv_path="notebook/.env", override=True)
import sys
from importlib import reload
from ploutos.api.bank_api import BankApi, Bank
if 'api.bank_api' in sys.modules:
    reload(sys.modules['ploutos.api.bank_api'])
else:
    print("BankApi module not loaded yet - no need to reload")


BankApi module not loaded yet - no need to reload


### Sheet Migration

In [ ]:
from ploutos.api.bank_api import BankApi, Bank
if 'db.models' in sys.modules:
    reload(sys.modules['db.models'])
else:
    print("db.models module not loaded yet - no need to reload")

from db.models import *
#account_api=BankApi(Bank.LCL)

#print(account_api.get_metadata())

db.models module not loaded yet - no need to reload


In [6]:
# sheet_migration=SheetMigration()

# print(sheet_migration.account_list)


In [ ]:
df_paiement=sheet_migration.get_data("Paiement")
df_transfert=sheet_migration.get_data("Transfert")
df_benefices=sheet_migration.get_data("Bénéfices")


In [ ]:
accounts_to_create_budget=[]
for index, row in df_paiement[['Catégorie','Sous Catégorie']].drop_duplicates().iterrows():
    accounts_to_create_budget.append(AccountCreate(
        category=row['Catégorie'] or "Unknown",
        sub_category="Unknown",
        is_real=False,
        name=row['Sous Catégorie'] or "Unknown",
        original_amount=0
    ))
print("Budget accounts to create: ",len(accounts_to_create_budget))

In [ ]:
accounts_to_creat_real=[]
for index, row in df_paiement[['Compte']].drop_duplicates().iterrows():
    accounts_to_creat_real.append(AccountCreate(
        category=row['Compte'] or "Unknown",
        sub_category="Unknown",
        is_real=True,
        name=row['Compte'] or "Unknown",
        original_amount=0
    ))
print("Real accounts to create: ",len(accounts_to_creat_real))


In [ ]:
accounts_to_create_missing=[]
for account in set(df_transfert['Compte départ']).union(set(df_transfert['Compte arrivée'])).union(set(df_benefices['Compte'])) -set(df_paiement['Compte']):
    accounts_to_create_missing.append(AccountCreate(
        category=account or "Unknown",
        sub_category="Unknown",
        is_real=True,
        name=account or "Unknown",
        original_amount=0
    ))
accounts_to_create_missing.append(AccountCreate(
        category="Unknown",
        sub_category="Unknown",
        is_real=True,
        name="Unknown",
        original_amount=0
    ))
print("Transfer and benefit accounts to create",len(accounts_to_create_missing))



In [8]:
from api.deps import get_db
from api.routers.accounts import *
import importlib
from api.routers.accounts import create_account
importlib.reload(importlib.import_module('api.routers.accounts'))


from fastapi import Depends
import asyncio
import nest_asyncio

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Create accounts from sheet data
async def create_accounts_from_sheet(accounts_to_create = []):
    db = get_db
    
    # Define account types based on sheet data
    
    # Create accounts
    for account in accounts_to_create:
        try:
            await create_account(account, db)
            print(f"Created account: {account.name}")
        except Exception as e:
            print(f"Error creating account {account.name}: {str(e)}")

# Run the account creation using asyncio.run() since we're using nest_asyncio
asyncio.run(create_accounts_from_sheet(accounts_to_creat_real+accounts_to_create_budget+accounts_to_create_missing))


NameError: name 'accounts_to_creat_real' is not defined

In [7]:
from api.routers.accounts import get_accounts
from api.deps import get_db

async def query_all_accounts():
    db = get_db
    accounts = await get_accounts(db)
    return accounts

# Run the query using asyncio.run() since we're using nest_asyncio
accounts = asyncio.run(query_all_accounts())
print("Found accounts:", len(accounts))
account_text=[accounts['name'] for accounts in accounts]
for account in accounts:
    account=Account(**account)
    print(f"- {account.name} (ID: {account.accountId})")


NameError: name 'asyncio' is not defined

In [ ]:
db = get_db
accounts = (
    db.table("Accounts")
    .select("*")
    .execute()
)
accounts = pd.DataFrame(accounts.data)


In [ ]:
from tqdm import tqdm
df_paiement['account_credit'] = None
df_paiement['account_debit'] = None

for index, row in tqdm(df_paiement.iterrows(), total=len(df_paiement)):
    try:
        debit_account = accounts[accounts['name'] == (row["Compte"] if pd.notna(row["Compte"]) else "Unknown")]
        if not debit_account.empty:
            df_paiement.at[index, 'account_debit'] = debit_account.iloc[0]['accountId']
        else:
            print(f"Warning: No matching debit account found for {row['Compte']}")
            
        credit_account = accounts[
            (accounts['name'] == (row["Sous Catégorie"] if row["Sous Catégorie"] else "Unknown")) &
            (accounts['category'] == (row["Catégorie"] if row["Catégorie"] else "Unknown")) & (accounts['is_real']==False)
        ]
        if not credit_account.empty:
            df_paiement.at[index, 'account_credit'] = credit_account.iloc[0]['accountId']
        else:
            print(f"Warning: No matching credit account found for {index}")
    except Exception as e:
        raise e
        print(f"Error getting account IDs for row {index}: {str(e)}")

In [ ]:
print(df_paiement[df_paiement["account_credit"].isna()])
print(df_paiement[df_paiement["account_debit"].isna()])



In [ ]:
import importlib
import db.models
importlib.reload(db.models)

from db.models import TransactionCreate, TransactionSlaveCreate
from db.migrations import upload_transactions

In [ ]:
from uuid import UUID, uuid4
from datetime import datetime
master_transactions = []
slaves_transactions = []
for index, row in df_paiement.iterrows():
    # Get account type ID for this transaction
    account_credit = row["account_credit"]
    account_debit = row["account_debit"]
    date=pd.Timestamp(datetime.now().timestamp())
    creditId=uuid4()
    debitId=uuid4()
    if row['Montant']>0:
        master_transactions.append(Transaction(
            transactionId=debitId,
            created_at=date,
            updated_at=date,
            description=row["Description"],
            date=pd.Timestamp(row["Date"]),
            type="debit",
            amount=row["Montant"],
            accountId=account_debit
    ))
        slaves_transactions.append(TransactionSlave(
            slaveId=creditId,
            created_at=date,
            updated_at=date,
            type="debit",
            amount=row["Montant"],
            date=pd.Timestamp(row["Date"]),
            accountId=account_credit,
            masterId=debitId,
                ))
    else:
        raise ValueError(f"Montant négatif pour la transaction {row}")



In [ ]:
df_benefices=df_benefices.merge(accounts,how='left',left_on='Compte',right_on='name')
df_transfert=df_transfert.merge(accounts[['accountId','name']].rename(columns={'accountId':'departID'}), how='left', left_on='Compte départ', right_on='name').merge(accounts[['accountId','name']].rename(columns={'accountId':'arriveeID'}), how='left', left_on='Compte arrivée', right_on='name')

In [ ]:
unknown_id = accounts[(accounts['sub_category'] == 'Unknown') & (accounts['category'] == 'Unknown') & (accounts['is_real'] == False) & (accounts['name'] == 'Unknown')]['accountId'].iloc[0]
unknown_id

In [ ]:
for index, row in df_benefices.iterrows():
    # Get account type ID for this transaction
    account_credit = row["accountId"]
    account_debit = unknown_id
    date=pd.Timestamp(datetime.now().timestamp())
    creditId=uuid4()
    debitId=uuid4()
    if row['Montant']>0:
        master_transactions.append(Transaction(
            transactionId=creditId,
            created_at=date,
            updated_at=date,
            description=row["Raison"],
            date=pd.Timestamp(row["Date"]),
            type="credit",
            amount=row["Montant"],
            accountId=account_credit
    ))
        slaves_transactions.append(TransactionSlave(
            slaveId=debitId,
            created_at=date,
            updated_at=date,
            type="debit",
            amount=row["Montant"],
            date=pd.Timestamp(row["Date"]),
            accountId=account_debit,
            masterId=creditId,
                ))
    else:
        raise ValueError(f"Montant négatif pour la transaction {row}")

In [ ]:
for index, row in df_transfert.iterrows():
    # Get account type ID for this transaction
    account_credit = row["arriveeID"]
    account_debit = row['departID']
    date=pd.Timestamp(datetime.now().timestamp())
    creditId=uuid4()
    debitId=uuid4()
    if row['Montant']>0:
        master_transactions.append(Transaction(
            transactionId=debitId,
            created_at=date,
            updated_at=date,
            description=row["Description"],
            date=pd.Timestamp(row["Date"]),
            type="debit",
            amount=row["Montant"],
            accountId=account_debit
    ))
        slaves_transactions.append(TransactionSlave(
            slaveId=creditId,
            created_at=date,
            updated_at=date,
            type="credit",
            amount=row["Montant"],
            date=pd.Timestamp(row["Date"]),
            accountId=account_credit,
            masterId=debitId,
                ))
    else:
        raise ValueError(f"Montant négatif pour la transaction {row}")

In [ ]:
assert len(master_transactions)==1581

In [ ]:

        

# Call the function with the database session and lists
upload_transactions(get_db, master_transactions, slaves_transactions)


In [ ]:
accounts

In [ ]:
from uuid import UUID, uuid4
from datetime import datetime
master_transactions = []
slaves_transactions = []
for index, row in df_paiement.iterrows():
    # Get account type ID for this transaction
    account_credit = row["account_credit"]
    account_debit = row["account_debit"]
    date=pd.Timestamp(datetime.now().timestamp())
    creditId=uuid4()
    debitId=uuid4()
    if row['Montant']>0:
        master_transactions.append(Transaction(
            transactionId=creditId,
            created_at=date,
            updated_at=date,
            description=row["Description"],
            date=pd.Timestamp(row["Date"]),
            type="debit",
            amount=-row["Montant"],
            accountId=account_debit
    ))
        slaves_transactions.append(TransactionSlave(
            slaveId=debitId,
            created_at=date,
            updated_at=date,
            type="debit",
            amount=row["Montant"],
            date=pd.Timestamp(row["Date"]),
            accountId=account_debit,
            masterId=creditId,
                ))
    else:
        raise ValueError(f"Montant négatif pour la transaction {row}")

### LCL

#### Compte Courant

In [ ]:
from ploutos.utils.scraping.scrap_lcl import convert_LCLpdf_to_df

old_lcl=convert_LCLpdf_to_df("ploutos/data/lcl/compte_courant")

In [ ]:
def calculate_last_transaction(db_payement, db_transfert, db_benefices,account):
    last_transaction=[]
    last_transaction.append(db_payement[db_payement["Compte"]==account]["Date"].max())
    last_transaction.append(db_transfert[db_transfert["Compte arrivée"]==account]["Date"].max())
    last_transaction.append(db_transfert[db_transfert["Compte départ"]==account]["Date"].max())
    last_transaction.append(db_benefices[db_benefices["Compte"]==account]["Date"].max())
    last_transaction=[ts for ts in last_transaction if pd.notna(ts)]
    if len(last_transaction)>0:
        last_transaction=max(last_transaction)
    else:
        last_transaction=None
    return last_transaction


In [ ]:
last_transaction=calculate_last_transaction(df_paiement, df_transfert, df_benefices,"Compte Courant").strftime("%Y-%m-%d")
transactions=account_api.get_transactions(date_from='2024-06-06')
print(last_transaction)


In [ ]:
debit_old=old_lcl[(old_lcl["Date"] <= "2024-09-30")&(old_lcl["Date"] > last_transaction)]['DEBIT'].sum()
credit_old=old_lcl[(old_lcl["Date"] <= "2024-09-30")&(old_lcl["Date"] > last_transaction)]['CREDIT'].sum()
sheet_value=float(sheet_migration.account_list[sheet_migration.account_list["Compte"]=="Compte Courant"]["Actuel"].values[0].replace("€","").replace(",","."))
end_september_balance=float(sheet_value)+credit_old-debit_old
print(f"End of September balance: {end_september_balance}")
current_balance=end_september_balance+transactions[transactions["Date"]>'2024-09-30']['Montant'].sum()
print(f"Current balance: {current_balance}")
official_balance=account_api.get_balance()
print(f"Official balance: {official_balance}")


#### Compte Epargne


Le livret jeune n'a pas subi de changement et va fermer. On regarde uniquement le livret A.
On détecte les transactions de M JEREMY BON dans le compte courant avec `(transactions["Description"].str.contains("M JEREMY BON"))`

In [ ]:
df_transfert[df_transfert["Compte départ"]=="Livret A"]

In [ ]:
convert_LCLpdf_to_df("ploutos/data/lcl/compte_livret")
last_transaction=calculate_last_transaction(df_paiement, df_transfert, df_benefices,"Livret A")
print(last_transaction)


In [ ]:
debit_old=old_lcl[(old_lcl["Description"].str.contains("M JEREMY BON"))&(old_lcl["Date"] <= "2024-09-30")&(old_lcl["Date"] > last_transaction)]['DEBIT'].sum()
credit_old=old_lcl[(old_lcl["Description"].str.contains("M JEREMY BON"))&(old_lcl["Date"] <= "2024-09-30")&(old_lcl["Date"] > last_transaction)]['CREDIT'].sum()
sheet_value=float(sheet_migration.account_list[sheet_migration.account_list["Compte"]=="Livret A"]["Actuel"].values[0].replace("€","").replace(",",".").replace("\u202f",""))
end_september_balance=float(sheet_value)-credit_old+debit_old
print(f"End of September balance: {end_september_balance}")
current_balance=end_september_balance-transactions[(transactions["Description"].str.contains("M JEREMY BON"))&(transactions["Date"]>'2024-09-30')]['Montant'].sum()
print(f"Current balance: {current_balance}")
official_balance=8000
print(f"Official balance: {official_balance}")

### Lydia

In [ ]:
last_transaction=calculate_last_transaction(df_paiement, df_transfert, df_benefices,"Lydia")
print(last_transaction)
sheet_value=float(sheet_migration.account_list[sheet_migration.account_list["Compte"]=="Lydia"]["Actuel"].values[0].replace("€","").replace(",",".").replace("\u202f",""))
print(f"Lydia balance on {last_transaction.strftime('%Y-%m-%d')}: {sheet_value}")


In [ ]:
lydia_account_value=50.84
new_transaction_lydia=pd.DataFrame([{
    "Date":"2024-09-01",
    "Libellé":"Madrid Gael Terier",
    "Montant":38.89,
    "Compte":"Lydia",
},
{
    "Date":"2024-08-31",
    "Libellé":"Guilhem Leborgne",
    "Montant": -4,
    "Compte":"Lydia",
    
},
{
    "Date":"2024-08-27",
    "Libellé":"Dépôt crte revolut 7255",
    "Montant": 25,
    "Compte":"Lydia",
},{
    "Date":"2024-08-27",
    "Libellé":"Guilhem Leborgne",
    "Montant": -25,
    "Compte":"Lydia",
},
{
    "Date":"2024-07-25",
    "Libellé":"Kdo Juan Lucie Formery",
    "Montant": -8.33,
    "Compte":"Lydia",
},
{
    "Date":"2024-07-25",
    "Libellé":"Juanita PA",
    "Montant": -3.33,
    "Compte":"Lydia",
},
{
    "Date":"2024-07-16",
    "Libellé":"Tren Gijon Lucie Formery",
    "Montant": 27.1,
    "Compte":"Lydia",
},
{
    "Date":"2024-07-01",
    "Libellé":"Thai Bastien Englebert",
    "Montant": -7.9,
    "Compte":"Lydia",
}
])
assert new_transaction_lydia['Montant'].sum()+sheet_value==lydia_account_value



### Revolut


In [ ]:
cd c:\Users\bonje\Documents\Perso\ploutos


In [ ]:

from ploutos.api.bank_api import BankApi, Bank


In [ ]:
lcl_api.get_transactions()

In [ ]:
lcl_api=BankApi(Bank.LCL)
lcl_api.api.get_transactions()


In [ ]:
revolut_api=BankApi(Bank.REVOLUT)


In [ ]:
raw_transac=revolut_api.api.get_transactions()

In [ ]:
pd.DataFrame(raw_transac['transactions']['pending'])

In [ ]:
transac_rev=revolut_api.get_transactions()


### Autres

In [ ]:
transac_rev

In [ ]:
sheet_migration.account_list

    ### Tricount (A faire)

### ticket resto (A solde)

#### Natixis Interépargne 


### Retrieve previous transactions

In [1]:
cd /Users/jeremy_bon/Documents/Perso/Ploutos/ploutos


/Users/jeremy_bon/Documents/Perso/Ploutos/ploutos


/Users/jeremy_bon/Library/Caches/pypoetry/virtualenvs/ploutos-syHIIjPM-py3.12/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [29]:
from api.deps import get_db
from api.routers.accounts import *
import importlib
import pandas as pd
importlib.reload(importlib.import_module('api.routers.accounts'))
from api.routers.accounts import get_current_amounts
from fastapi import Depends
import nest_asyncio
from api.bank_api import BankApi


In [42]:
from api.routers.accounts import get_current_amounts

async def query_all_accounts():
    accounts = await get_current_amounts(get_db)
    return accounts

# If running in Jupyter, use nest_asyncio and run the coroutine directly
accounts = await query_all_accounts()
df_accounts = pd.DataFrame([a.model_dump() for a in accounts])

df_accounts.head(2)

,account_id,name,category,sub_category,current_amount,is_real,max_date
0,42aa4dec-ae44-4898-a0f5-17b063283314,Unknown,Unknown,Unknown,0.000000e+00,True,NaT
1,e775ea39-f089-4432-83c0-f2477aba9248,Lydia,Lydia,Unknown,2.999379e-12,True,2025-06-23 13:10:44.393768+00:00


In [61]:
account_id,current_amount=df_accounts[df_accounts['name'].str.contains("LCL")][['account_id','current_amount']].iloc[0]

In [66]:
account_api=BankApi(accountId=account_id)

Error getting metadata for LCL_CRLYFRPP: {'response': {'summary': 'No Account matches the given query.', 'detail': 'No Account matches the given query.', 'status_code': 404}, 'status': 404}


HTTPError: {'response': {'summary': 'No Account matches the given query.', 'detail': 'No Account matches the given query.', 'status_code': 404}, 'status': 404}

### processus pour faire un id qui marche

In [49]:
import importlib
import pandas as pd
importlib.reload(importlib.import_module('api.bank_api'))
from api.bank_api import connect_to_bank

connect_to_bank("LCL_CRLYFRPP",requisition_id='645d79b0-c29d-47b0-9fdd-c83b9b15e807')

2025-10-16 23:05:54.053 | INFO     | api.bank_api:connect_to_bank:35 - Requisition ID:


{'id': '645d79b0-c29d-47b0-9fdd-c83b9b15e807',
 'created': '2025-10-16T20:39:51.302602Z',
 'redirect': 'https://gocardless.com',
 'status': 'LN',
 'institution_id': 'LCL_CRLYFRPP',
 'agreement': 'e8d4935c-9453-41f2-b644-702b3cd7a179',
 'reference': '46341f98-572d-45d2-9120-e092a9af64b8',
 'accounts': ['1bdb6174-5522-44d5-a67e-28d50123966f'],
 'link': 'https://ob.gocardless.com/ob-psd2/start/0507a7ca-4f8e-47ac-b1b9-1495a1eb4750/LCL_CRLYFRPP',
 'ssn': None,
 'account_selection': False,
 'redirect_immediate': False}

In [67]:
client.account_api(id="1bdb6174-5522-44d5-a67e-28d50123966f").get_balances()

{'balances': [{'balanceAmount': {'amount': '-446.98', 'currency': 'EUR'},
   'balanceType': 'expected',
   'referenceDate': '2025-10-16'},
  {'balanceAmount': {'amount': '1543.02', 'currency': 'EUR'},
   'balanceType': 'closingBooked',
   'referenceDate': '2025-10-15'}]}

In [84]:
account_id,current_amount=df_accounts[df_accounts['name'].str.contains("LCL")][['account_id','current_amount']].iloc[0]

In [85]:
account_id

'5e139075-c4b5-4f6f-afdb-e8a5efa9d377'

In [92]:
importlib.reload(importlib.import_module('api.bank_api'))
importlib.reload(importlib.import_module('utils.secrets'))
from api.bank_api import BankApi
account_api=BankApi(accountId=account_id)

Successfully connected to LCL_CRLYFRPP


In [93]:
from utils.secrets import save_secret
from db.models import AccountsSecretsCreate
from datetime import datetime
account=AccountsSecretsCreate(  
    accountId=account_id,
    bankId="LCL_CRLYFRPP",
    updated_at=datetime.now().isoformat(),
    secretId="1bdb6174-5522-44d5-a67e-28d50123966f")
save_secret(account)

In [10]:
api.get_metadata()

HTTPError: {'response': {'summary': 'Authentication failed', 'detail': 'Authentication credentials were not provided.', 'status_code': 401}, 'status': 401}

In [8]:
from uuid import UUID, uuid4
init = client.initialize_session(
    # institution id
    institution_id='LCL_CRLYFRPP',
    # redirect url after successful authentication
    redirect_uri="https://gocardless.com",
    # additional layer of unique ID defined by you
    reference_id=str(uuid4())
)
link = init.link # bank authorization link
requisition_id = init.requisition_id
link
#bien valider le link

HTTPError: {'response': {'summary': 'Authentication failed', 'detail': 'Authentication credentials were not provided.', 'status_code': 401}, 'status': 401}

In [9]:
import secrets
key = secrets.token_bytes(32)  # 32 bytes = 256 bits pour AES-256
print(key.hex())

31a37627ea3a368dbc5319aaefb1732757bbe7a7e40f0d8756f2e2a51c8b91fe


In [15]:
client.requisition.get_requisition_by_id(
    requisition_id="25f6b579-26ab-4be6-9786-f4a3182391f4"
)

HTTPError: {'response': {'summary': 'No Requisition matches the given query.', 'detail': 'No Requisition matches the given query.', 'status_code': 404}, 'status': 404}

In [ ]:
accounts["accounts"] # les id des différents comptes

['a52da9b8-2983-4428-bfe9-e1e2c85dbcb7',
 'f47d8c6f-6bca-41ae-9469-b4f88ecb2e61']